<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D8%A7%D9%84%D9%87%D9%8A%D8%AF%D8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def hedge_position_with_retrace_limit(symbol):
    """تطبيق استراتيجية الهيدج مع قيود على عدد الصفقات المفتوحة وحجم الصفقة الجديدة بنفس الاتجاه."""
    positions = mt5.positions_get(symbol=symbol)
    open_positions_count = len(positions)

    # تحقق من عدد الصفقات المفتوحة
    if open_positions_count >= 7:
        print("عدد الصفقات المفتوحة بلغ الحد الأقصى (7).")
        return

    if positions:
        for position in positions:
            # تحديد سعر الدخول ووقف الخسارة
            entry_price = position.price_open
            stop_loss = position.sl  # سعر وقف الخسارة

            # حساب المسافة بين سعر الدخول ووقف الخسارة
            distance_to_sl = entry_price - stop_loss if position.type == 0 else stop_loss - entry_price  # 0 = BUY, 1 = SELL

            # حساب السعر الذي يمثل 75% من المسافة نحو وقف الخسارة
            retrace_price = entry_price - (0.75 * distance_to_sl) if position.type == 0 else entry_price + (0.75 * distance_to_sl)  # 0 = BUY, 1 = SELL

            # الحصول على السعر الحالي
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask  # 0 = BUY, 1 = SELL

            # إذا كان السعر الحالي قد تراجع بنسبة 75% نحو وقف الخسارة
            if (position.type == 0 and current_price <= retrace_price) or \
               (position.type == 1 and current_price >= retrace_price):  # 0 = BUY, 1 = SELL

                # فتح مركز بنفس الاتجاه
                hedge_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": position.volume,  # نفس حجم المركز الحالي
                    "type": position.type,  # نفس نوع الصفقة (0 = BUY, 1 = SELL)
                    "price": mt5.symbol_info_tick(symbol).ask if position.type == 0 else mt5.symbol_info_tick(symbol).bid,
                    "sl": 0,
                    "tp": 0,
                    "magic": 123456,
                    "comment": "Hedge position in the same direction",
                }

                # إرسال الطلب
                result = mt5.order_send(hedge_request)
                if result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(f"فشل في فتح الصفقة: {result.retcode}")
